## Noise adding for regularization

In [1]:
import math
import numpy as np
import pandas as pd



For Data Preprocessing, first we deal with anomaly data, basically data with wrong or invalid format.

In [2]:
def readdata(data):
    
	# 把有些數字後面的奇怪符號刪除
	for col in list(data.columns[2:]):
		data[col] = data[col].astype(str).map(lambda x: x.rstrip('x*#A'))
	data = data.values
	
	# 刪除欄位名稱及日期
	data = np.delete(data, [0,1], 1)
	
	# 特殊值補0
	data[ data == 'NR'] = 0
	data[ data == ''] = 0
	data[ data == 'nan'] = 0
	data = data.astype(np.float)

	return data

We flatten our data to be in such format (col: one hour/ per col, row: one feature/ per row)

In [3]:
def extract(data):
	N = data.shape[0] // 18

	temp = data[:18, :]
    
    # Shape 會變成 (x, 18) x = 取多少hours
	for i in range(1, N):
		temp = np.hstack((temp, data[i*18: i*18+18, :]))
	return temp

Since some data points (PM2.5) have anomaly values, which strongly effect our training result, we decide to abandon them. In our case, we define PM2.5 < 2 or > 100 as anomaly data points. 

In [4]:
def valid(x, y):
	if y <= 2 or y > 100:
		return False
	for i in range(9):
		if x[9,i] <= 2 or x[9,i] > 100:
			return False
	return True

def parse2train(data):
	x = []
	y = []
	
	# 用前面9筆資料預測下一筆PM2.5 所以需要-9
	total_length = data.shape[1] - 9
	for i in range(total_length):
		x_tmp = data[:,i:i+9]
		y_tmp = data[9,i+9]
		if valid(x_tmp, y_tmp):
			x.append(x_tmp.reshape(-1,))
			y.append(y_tmp)
	# x 會是一個(n, 18, 9)的陣列， y 則是(n, 1) 
	x = np.array(x)
	y = np.array(y)
	return x,y

def parse2test(data):
	x = []
	
	total_length = data.shape[1] // 9
	for i in range(total_length):
		x_tmp = data[:,i*9:i*9+9]
		x.append(x_tmp.reshape(-1,))
	# x 會是一個(n, 18, 9)的陣列， y 則是(n, 1) 
	x = np.array(x)
	return x

This is our gradient descent algorithm. **Adam** was implemented.

In [5]:
def minibatch(x, y):
    # 打亂data順序
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    
    # 訓練參數以及初始化
    batch_size = 64
    lr = 1e-3
    lam = 0.001
    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    w = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    
    for num in range(1000):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b*batch_size:(b+1)*batch_size]
            y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)
            loss = y_batch - np.dot(x_batch,w) - bias
            
            # 計算gradient
            g_t = np.dot(x_batch.transpose(),loss) * (-2) +  2 * lam * np.sum(w)
            g_t_b = loss.sum(axis=0) * (2)
            m_t = beta_1*m_t + (1-beta_1)*g_t 
            v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
            m_cap = m_t/(1-(beta_1**t))
            v_cap = v_t/(1-(beta_2**t))
            m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
            v_t_b = 0.99*v_t_b + (1-0.99)*(g_t_b*g_t_b) 
            m_cap_b = m_t_b/(1-(0.9**t))
            v_cap_b = v_t_b/(1-(0.99**t))
            w_0 = np.copy(w)
            
            # 更新weight, bias
            w -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
            bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)
            

    return w, bias

In [6]:
def read_once(path):
    year1_pd = pd.read_csv(path)
    year1 = readdata(year1_pd)
    train_data = extract(year1)
    return parse2train(train_data)
    

def read_once2(path1, path2):
    # read y1
    year1_pd = pd.read_csv(path1)
    year1 = readdata(year1_pd)
    train_data = extract(year1)
    train_x, train_y = parse2train(train_data)
    # Read y2
    year2_pd = pd.read_csv(path2)    
    year2 = readdata(year2_pd)
    train_data2 = extract(year2)
    train_x2, train_y2 = parse2train(train_data2)
    
    # concate
    train_x = np.vstack((train_x,train_x2))
    train_y = np.concatenate((train_y,train_y2))
    return train_x, train_y
# **Combine them together!**

def read_test(path3):
    testing_pd = pd.read_csv(path3)
    testing = readdata(testing_pd)
    testing_data = extract(testing)
    return parse2test(testing_data)

def read_item_list(path):
    from collections import OrderedDict
    year1_pd = pd.read_csv(path)
    return list(OrderedDict.fromkeys(year1_pd.iloc[::,1]))

# Read data

In [7]:
np.random.seed(0)
path = './data/year2-data.csv'
item_list = read_item_list(path)

year_pd = pd.read_csv(path)
year = readdata(year_pd)
train_data = extract(year)

# Adding noise in data as regularization

### Case 1:抽全部9小時內的污染源feature當作一次項(加bias)
Adding noise before parsing.

Noise is in raw data not in features!

# Wrong code in noise level but better result

In [8]:
[np.var(train_data[i,::]) for i in range(len(train_data))]

[38.43047691861617,
 0.027521032583765977,
 0.059790400636715244,
 0.01164623556118513,
 36.49385222355142,
 102.65542733666418,
 202.25939268103252,
 369.1745158769626,
 590.4693430078604,
 7717.303667550197,
 4.2946946263630865,
 197.6025015319739,
 5.525599986317008,
 0.05338167208773795,
 6141.835352129335,
 6413.486185131539,
 1.0198066240017931,
 0.8722428033975521]

In [9]:
# Add noise here
noise_level = 0.1
train_data.shape
for i in range(len(train_data)):
# wrong code but better result
#     train_data[i] = train_data[i] + np.random.normal(0, np.var(train_data[0,::])*noise_level, len(train_data[i]))
# right one
    if i != 9:
        train_data[i] = train_data[i] + np.random.normal(0, (np.var(train_data[i,::])*noise_level)**(1/2), len(train_data[i]))
    else:
        train_data[i] = train_data[i] + np.random.normal(0, np.var(train_data[0,::])*noise_level, len(train_data[i]))
train_x, train_y = parse2train(train_data)
train_x.shape

(5726, 162)

In [10]:
[np.var(train_data[i,::]) for i in range(len(train_data))]

[42.43839518200506,
 0.030007787230198476,
 0.06566074305586862,
 0.012849231038008033,
 40.05686951719266,
 112.25147232075302,
 222.51666993065038,
 406.5096374934065,
 647.8534882543198,
 7742.752825652213,
 4.732297075460375,
 218.97288099748417,
 6.110712135370229,
 0.05839445618139722,
 6699.993985166474,
 7021.502590797796,
 1.1160657488873078,
 0.9578207709381813]

In [11]:
# training
w, bias = minibatch(train_x, train_y)

np.save('best_w.npy', w)
np.save('best_bias.npy',bias)

# read testing and pred
# test_path = './data/testing_data.csv'
# test_x = read_test(test_path)

# pred_y = test_x @ w + bias
# id_list = [ "id_"+str(i) for i in range(len(test_x))]
# output_pd = pd.DataFrame(zip(id_list, pred_y.ravel()), columns=['id', 'value'])

# output_name = './output/y2_data_noise0.1_dataSel_noiseReg.csv'
# output_pd.to_csv(output_name, index=False)

# train_y_pred = train_x @ w + bias
# err = np.mean((train_y_pred.ravel() - train_y.ravel())**2)
# err

# Not yet finished

# Column selection by cross validation